# Block 7 Übung 5: Exploration von Messdaten gegenüber Diagnosen

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "tidyr", "icd.data", "ggplot2", "stringr")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"

## MIMIC III-Basisdaten laden

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Diagnosen laden & Bezeichner ergänzen

In [ ]:
# Diagnosen laden
mimic.diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/DIAGNOSES_ICD.csv", sep=""),
                                col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), seq_num = col_integer(), icd9_code = col_character()))

# Diagnosebezeichner laden
mimic.d_icd_diagnoses.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ICD_DIAGNOSES.csv", sep=""),
                                      col_types = cols(row_id = col_integer(), icd9_code = col_character(), short_title = col_character(), long_title = col_character()))

# Diagnosebezeichner hinzufügen
mimic.diagnoses.annotated <- mimic.diagnoses.raw %>% inner_join(mimic.d_icd_diagnoses.raw %>% select(icd9_code, short_title, long_title), by = "icd9_code")

## MIMIC III-Labordaten laden & Bezeichner ergänzen

In [ ]:
# Laborbefunde laden
mimic.labevents.raw <- read_csv(paste(base_url, "mimic-iii-demo/LABEVENTS.csv", sep=""),
                                col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), itemid = col_integer(), charttime = col_datetime(format = ""), value = col_character(), valuenum = col_double(), valueuom = col_character(), flag = col_character()))

# Bezeichner laden
mimic.d_labitems.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_LABITEMS.csv", sep=""),
                                 col_types = cols(row_id = col_integer(), itemid = col_integer(), label = col_character(), fluid = col_character(), category = col_character(), loinc_code = col_character()))

# Bezeichner hinzufügen
mimic.labevents.annotated <- mimic.labevents.raw %>% 
  inner_join(mimic.d_labitems.raw %>% select(itemid, label, fluid, category, loinc_code), by = "itemid")

## ICD9-Diagnosekatalog (inkl. Kapitel-Hierarchie) laden

In [ ]:
# ICD9-Hierarchie aus Package in ein Tibble überführen
icd9.hierarchy.raw <- icd9cm_hierarchy

# Datentypanpassung der "code"-Spalte, um einen Join mit den MIMIC-Diagnosen zu ermöglichen
icd9.hierarchy.raw$icd9_code <- as.character(icd9.hierarchy.raw$code)

## Hauptdiagnosen auf ICD9-Kapitel abbilden und den Labordaten zuordnen

Hinweis: Laut Dokumentation wird die Hauptdiagnose in der Regel "als erstes" angegeben und ist über die Spalte seq_num == 1 selektierbar

In [ ]:
diagnoses_labs <- mimic.diagnoses.raw %>% 
    # Hauptdiagnose filtern
    filter(seq_num == 1) %>%
    # nur Spalten Fallnummer und ICD9-Code selektieren
    select(hadm_id, icd9_code) %>%
    # ICD9-Kapitel aus Hierarchie ergänzen
    inner_join(icd9.hierarchy.raw %>% select(icd9_code, chapter), by = "icd9_code") %>%
    # Mehrfachdiagnosen entfernen
    distinct() %>%
    # Relevante Spalten der Laborbefunde hinzufügen
    inner_join(mimic.labevents.annotated %>% select(hadm_id, loinc_code, label, fluid, valuenum, valueuom), by = "hadm_id")
head(diagnoses_labs)

## Laboranalyte für Kreatinin heraussuchen

In [ ]:
# Verfügbare Analyte & Befunde zum Kreatinin anzeigen
mimic.labevents.annotated %>%
    filter(grepl("Creatinine", label)) %>%
    group_by(label, fluid, loinc_code) %>%
    summarize(n = n(), .groups = "keep")

## Kreatininbefunde gegenüber den ICD9-Kapiteln darstellen

Hinweise:
* es gibt einen Ausreißer, dessen Befund eher in den Wertebereich von Kreatininmessungen im Urin fällt (Fehldokumentation?); dieser wird durch ein entsprechendes Filterkriterium im data-segment der `ggplot()`-Funktion entfernt. **Wichtig**: nicht vergessen, dieses Filterkriterium bei der Wiederverwendung des Codefragments für andere Analyte herauszunehmen oder ggf. anzupassen.
* mit str `str_wrap()`-Funktion aus dem Package `stringr`können lange Bezeichner automatisch umgebrochen werden, um die Darstellung lesbarer zu machen b zw. den nötigen Platz für die Achsenbeschriftung/Legende zu reduzieren

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
ggplot(data = diagnoses_labs %>% filter(label == "Creatinine", 
                                        valuenum < 100),  # Ausfilterung eines einzelnen Ausreißers
       aes(x = str_wrap(chapter, 30), y = valuenum, fill = str_wrap(chapter, 30))) +
  geom_boxplot() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 8),
        legend.text = element_text(size = 8))

## Aufgabe: Finden & visualisieren Sie Laborbefunde, bei denen Sie Unterschiede zwischen den verschiedenen ICD9-Diagnosekapiteln erwarten würden!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) und dem Abschnitt 6, Demo 1 (Daten visualisieren, Diagramme) anwenden.
* zugehörige Notebooks: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb), [teil-6-demo-01-visualisieren-diagramme.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-6-demo-01-visualisieren-diagramme.ipynb)
* nutzen Sie gerne auch das [ggplot2-Cheat Sheet von RStudio](https://github.com/rstudio/cheatsheets/raw/master/data-visualization-2.1.pdf)

Hinweise:
* aufgrund der niedrigen Fallzahl & des komplexen Patientenguts (Intensivstation), gerade auch durch mehrere parallel bestehende Diagnosen, werden die Ergebnisse häufig eher heterogen ausfallen - es geht hier mehr um die grundsätzliche Vorgehensweise
* die Auswertung kann zusätzlich auch dadurch verzerrt sein, dass Laboruntersuchungen erkrankungs- & symptombezogen angefordert werden und daher ggf. bestimmte Analyte bei Patient:innen in anderen ICD9-Diagnosekapiteln nicht oder nur selten bestimmt werden
* es sollten deshalb Analyte ausgewählt werden, die häufig angefordert wurden